In [1]:
import pandas as pd
import numpy as np
import csv

In [2]:
def addName(fantasyTotal):
    #split player name and IDs in fantasyTotal dataframe
    playerName = []
    playerID = []
    for name in fantasyTotal['Player']:
        splitName = str(name).split('\\')
        if len(splitName[0]) > 0:
            playerName.append(splitName[0])
        else:
            playerName.append('NA')

        if len(splitName[1]) > 0:
            playerID.append(splitName[1])
        else:
            playerID.append('NA')        


    fantasyTotal['Name'] = playerName
    fantasyTotal['ID'] = playerID
    return fantasyTotal

In [3]:
def addInjury(fantasyTotal):
    
    #import report
    injury = pd.read_csv('InjuryReport.csv')
    
    #split names in injury report
    injuryName = []
    for name in injury['Player']:
        injuryName.append(name.split('\\')[0])

    injury['Name'] = injuryName
    
    #Add injuries to the fantasyTotal dataframe
    injList = []
    for name in fantasyTotal['Name']:
        status = injury[injury['Name'] == name]['Class']
        if len(status) > 0:
            injList.append(status.iloc[0])
        else:
            injList.append('NA')

    fantasyTotal['Injury'] = injList
    return fantasyTotal

In [4]:
def addSalary(fantasyTotal):
    
    #import salary list
    dkSalary = pd.read_csv('DKSalaries_Week9.csv')
    
    #add salaries to the fantasyTotal dataframe
    salaryList = []
    for player in fantasyTotal['Name']:
        sal = dkSalary[dkSalary['Name'] == player]['Salary']
        if len(sal) > 0:
            salaryList.append(sal.iloc[0])
        else:
            salaryList.append(0)

    fantasyTotal['Salary'] = salaryList
    
    return fantasyTotal

In [5]:
def dataCleanUp(fantasyTotal):
    #remove SNF and MNF games
    fantasyTotal = fantasyTotal[(fantasyTotal['Team'] != 'TAM') & (fantasyTotal['Team'] != 'NOR') & (fantasyTotal['Team'] != 'NWE') & (fantasyTotal['Team'] != 'NYJ')]
    #remove TNF
    fantasyTotal = fantasyTotal[(fantasyTotal['Team'] != 'GNB') & (fantasyTotal['Team'] != 'SFO')]
    #remove postponed teams - no postponed teams this week
    #fantasyTotal = fantasyTotal[(fantasyTotal['Team'] != 'TEN') & (fantasyTotal['Team'] != 'BUF') & (fantasyTotal['Team'] != 'NWE') & (fantasyTotal['Team'] != 'DEN')]
    #remove bye weeks
    fantasyTotal = fantasyTotal[(fantasyTotal['Team'] != 'CIN') & (fantasyTotal['Team'] != 'CLE') & (fantasyTotal['Team'] != 'LAR') & (fantasyTotal['Team'] != 'PHI')]
    #remove players that are out, probable, doubtful, and IR
    fantasyTotal = fantasyTotal[(fantasyTotal['Injury'] == 'NA') | (fantasyTotal['Injury'] == 'questionable')]
    #remove players that have played only 2 game
    #fantasyTotal = fantasyTotal[(fantasyTotal['G'] > 2)]
    
    #remove any players that have not scored points so far
    fantasyTotal['DKPt'] = fantasyTotal['DKPt'].fillna(0)
    fantasyTotal = fantasyTotal[fantasyTotal['DKPt'] > 0]
        
    return fantasyTotal

In [6]:
def issuesList(fantasyTotal):
    issueList = fantasyTotal[fantasyTotal['Salary'] == 0]
    
    issueList.to_csv('issuesList.csv')    
    return issueList

In [7]:
def issuesFix(fantasyTotal):
    playerName = []
    playerSalary = []
    issuesFix = pd.read_csv('issuesFix.csv')
    
    for name in fantasyTotal['Name']:
        sal = fantasyTotal[fantasyTotal['Name'] == name]['Salary'].iloc[0]
        for oldName in issuesFix['Name']:
            if name == oldName:
                newName = str(issuesFix[issuesFix['Name'] == oldName]['fixName'].iloc[0])
                newSalary = issuesFix[issuesFix['Name'] == oldName]['fixSalary'].iloc[0]
                name = newName
                sal = newSalary
        playerName.append(name)
        playerSalary.append(sal)
    
    fantasyTotal['Name'] = playerName
    fantasyTotal['Salary'] = playerSalary
    
    #filter to remove empty non-fixed players
    fantasyTotal = fantasyTotal[fantasyTotal['Salary'].notnull()]
    return fantasyTotal

In [8]:
def findSoS(fantasyTotal, week):
    #import necessary files
    teamDef = pd.read_csv('TeamDefense.csv')
    teamOff = pd.read_csv('TeamOffense.csv')
    schedule = pd.read_csv('Schedule.csv')
    teamID = pd.read_csv('conversion.csv')
    
    #add in team abbreviations to the schedule
    newSch = pd.DataFrame()
    newSch2 = pd.DataFrame()
    for team, abb in zip(teamID['Team'], teamID['Abbrev']):
        ind = schedule[schedule['Winner/tie'] == team]
        ind['Winner Team'] = abb
        newSch = newSch.append(ind)
    for team, abb in zip(teamID['Team'], teamID['Abbrev']):
        ind = newSch[newSch['Loser/tie'] == team]
        ind['Loser Team'] = abb
        newSch2 = newSch2.append(ind)
    schedule = newSch2
    
    #find standings for each team
    dummySchedule = schedule[schedule['Week'] == week]    
    preSch = schedule[schedule['Week'] != week]

    wins = 0
    winnerList = []
    loses = 0
    loserList = []
    for winner in dummySchedule['Winner/tie']:
        for winTeam in preSch['Winner/tie']:
            if winTeam == winner:
                wins = wins + 1
        for loseTeam in preSch['Loser/tie']:
            if loseTeam == winner:
                loses = loses + 1

        winnerList.append(wins)
        loserList.append(loses)

        wins = 0
        loses = 0

    dummySchedule['Winner Wins'] = winnerList
    dummySchedule['Winner Loses'] = loserList

    wins = 0
    winnerList = []
    loses = 0
    loserList = []
    for loser in dummySchedule['Loser/tie']:
        for winTeam in preSch['Winner/tie']:
            if winTeam == loser:
                wins = wins + 1
        for loseTeam in preSch['Loser/tie']:
            if loseTeam == loser:
                loses = loses + 1

        winnerList.append(wins)
        loserList.append(loses)

        wins = 0
        loses = 0

    dummySchedule['Loser Wins'] = winnerList
    dummySchedule['Loser Loses'] = loserList
    
    #find SoS
    totWins = 0
    totLoses = 0
    sosWinners = []
    sosLosers = []

    for winner in dummySchedule['Winner/tie']:
        dfWins = preSch[preSch['Winner/tie'] == winner]['Loser/tie']
        dfLoses = preSch[preSch['Loser/tie'] == winner]['Winner/tie']
        for team in dfWins:
            if team == 'Cincinnati Bengals':
                totWins = totWins + 2
                totLoses = totLoses + 5
            if team == 'Cleveland Browns':
                totWins = totWins + 5
                totLoses = totLoses + 3
            if team == 'Los Angeles Rams':
                totWins = totWins + 5
                totLoses = totLoses + 3
            if team == 'Philadelphia Eagles':
                totWins = totWins + 3
                totLoses = totLoses + 4
            if len(dummySchedule[dummySchedule['Winner/tie'] == team]) > 0:
                totWins = totWins + dummySchedule[dummySchedule['Winner/tie'] == team]['Winner Wins'].iloc[0]
                totLoses = totLoses + dummySchedule[dummySchedule['Winner/tie'] == team]['Winner Loses'].iloc[0]
            elif len(dummySchedule[dummySchedule['Loser/tie'] == team]) > 0:
                totWins = totWins + dummySchedule[dummySchedule['Loser/tie'] == team]['Loser Wins'].iloc[0]
                totLoses = totLoses + dummySchedule[dummySchedule['Loser/tie'] == team]['Loser Loses'].iloc[0]
        for team in dfLoses:
            if team == 'Cincinnati Bengals':
                totWins = totWins + 2
                totLoses = totLoses + 5
            if team == 'Cleveland Browns':
                totWins = totWins + 5
                totLoses = totLoses + 3
            if team == 'Los Angeles Rams':
                totWins = totWins + 5
                totLoses = totLoses + 3
            if team == 'Philadelphia Eagles':
                totWins = totWins + 3
                totLoses = totLoses + 4
            if len(dummySchedule[dummySchedule['Winner/tie'] == team]) > 0:
                totWins = totWins + dummySchedule[dummySchedule['Winner/tie'] == team]['Winner Wins'].iloc[0]
                totLoses = totLoses + dummySchedule[dummySchedule['Winner/tie'] == team]['Winner Loses'].iloc[0]
            elif len(dummySchedule[dummySchedule['Loser/tie'] == team]) > 0:
                totWins = totWins + dummySchedule[dummySchedule['Loser/tie'] == team]['Loser Wins'].iloc[0]
                totLoses = totLoses + dummySchedule[dummySchedule['Loser/tie'] == team]['Loser Loses'].iloc[0]
        sosWinners.append((totWins/(totWins+totLoses)))
        totWins = 0
        totLoses = 0

    for winner in dummySchedule['Loser/tie']:
        dfWins = preSch[preSch['Winner/tie'] == winner]['Loser/tie']
        dfLoses = preSch[preSch['Loser/tie'] == winner]['Winner/tie']
        for team in dfWins:
            if team == 'Cincinnati Bengals':
                totWins = totWins + 2
                totLoses = totLoses + 5
            if team == 'Cleveland Browns':
                totWins = totWins + 5
                totLoses = totLoses + 3
            if team == 'Los Angeles Rams':
                totWins = totWins + 5
                totLoses = totLoses + 3
            if team == 'Philadelphia Eagles':
                totWins = totWins + 3
                totLoses = totLoses + 4            
            if len(dummySchedule[dummySchedule['Winner/tie'] == team]) > 0:
                totWins = totWins + dummySchedule[dummySchedule['Winner/tie'] == team]['Winner Wins'].iloc[0]
                totLoses = totLoses + dummySchedule[dummySchedule['Winner/tie'] == team]['Winner Loses'].iloc[0]
            elif len(dummySchedule[dummySchedule['Loser/tie'] == team]) > 0:
                totWins = totWins + dummySchedule[dummySchedule['Loser/tie'] == team]['Loser Wins'].iloc[0]
                totLoses = totLoses + dummySchedule[dummySchedule['Loser/tie'] == team]['Loser Loses'].iloc[0]
        for team in dfLoses:
            if team == 'Cincinnati Bengals':
                totWins = totWins + 2
                totLoses = totLoses + 5
            if team == 'Cleveland Browns':
                totWins = totWins + 5
                totLoses = totLoses + 3
            if team == 'Los Angeles Rams':
                totWins = totWins + 5
                totLoses = totLoses + 3
            if team == 'Philadelphia Eagles':
                totWins = totWins + 3
                totLoses = totLoses + 4
            if len(dummySchedule[dummySchedule['Winner/tie'] == team]) > 0:
                totWins = totWins + dummySchedule[dummySchedule['Winner/tie'] == team]['Winner Wins'].iloc[0]
                totLoses = totLoses + dummySchedule[dummySchedule['Winner/tie'] == team]['Winner Loses'].iloc[0]
            elif len(dummySchedule[dummySchedule['Loser/tie'] == team]) > 0:
                totWins = totWins + dummySchedule[dummySchedule['Loser/tie'] == team]['Loser Wins'].iloc[0]
                totLoses = totLoses + dummySchedule[dummySchedule['Loser/tie'] == team]['Loser Loses'].iloc[0]
        sosLosers.append((totWins/(totWins+totLoses)))
        totWins = 0
        totLoses = 0    

    dummySchedule['Winner SoS'] = sosWinners
    dummySchedule['Loser SoS'] = sosLosers
    
    return dummySchedule

In [9]:
def scheduleAnalysis(fantasyTotal, schedule):

    #import necessary files
    teamDef = pd.read_csv('TeamDefense.csv')
    teamOff = pd.read_csv('TeamOffense.csv')

    #add reverse ranking
    teamOff['ReverseRank'] = teamOff['Rk'].apply(lambda x: (64-x))
    teamDef['ReverseRank'] = teamDef['Rk'].apply(lambda x: (32-x))
    
    #Add offense and defense rankings for each team in Winner column
    schDef = []
    rankD = []
    schOff = []
    rankO = []
    winTeam = []
    loseTeam = []
    for team in schedule['Winner/tie']:
        rnk = teamDef[teamDef['Tm'] == team]['Rk']
        refR = teamDef[teamDef['Tm'] == team]['ReverseRank']
        rnk2 = teamOff[teamOff['Tm'] == team]['Rk']
        refR2 = teamOff[teamOff['Tm'] == team]['ReverseRank'] 
        if len(rnk) > 0:
            schDef.append(int(rnk.iloc[0]))
            schOff.append(int(rnk2.iloc[0]))
            rankD.append(int(refR.iloc[0]))
            rankO.append(int(refR2.iloc[0]))
        else:
            schDef.append(0)
            schOff.append(0)

    schedule['Winner Defense Rank'] = schDef
    schedule['Winner Offense Rank'] = schOff
    schedule['ReverseRank Win Off'] = rankO
    schedule['ReverseRank Win Def'] = rankD

    #Add offense and defense rankings for each team in Loser column
    schDef = []
    schOff = []
    rankD = []
    rankO = []
    for team in schedule['Loser/tie']:
        rnk = teamDef[teamDef['Tm'] == team]['Rk']
        refR = teamDef[teamDef['Tm'] == team]['ReverseRank']
        rnk2 = teamOff[teamOff['Tm'] == team]['Rk']
        refR2 = teamOff[teamOff['Tm'] == team]['ReverseRank'] 
        if len(rnk) > 0:
            schDef.append(int(rnk.iloc[0]))
            schOff.append(int(rnk2.iloc[0]))
            rankD.append(int(refR.iloc[0]))
            rankO.append(int(refR2.iloc[0]))
        else:
            schDef.append(0)
            schOff.append(0)

    schedule['Loser Defense Rank'] = schDef
    schedule['Loser Offense Rank'] = schOff
    schedule['ReverseRank Lose Off'] = rankO
    schedule['ReverseRank Lose Def'] = rankD

    #Find deltas between opposing off/def
    schedule['WinOff/LoseDef Delta'] = schedule['Winner Offense Rank'] - schedule['Loser Defense Rank']
    schedule['LoseOff/WinDef Delta'] = schedule['Loser Offense Rank'] - schedule['Winner Defense Rank']
    
    schedule['RR LoseO/WinD'] = abs((schedule['ReverseRank Lose Off'] - schedule['ReverseRank Win Def'])**2)
    schedule['RR WinO/LoseD'] = abs((schedule['ReverseRank Win Off'] - schedule['ReverseRank Lose Def'])**2)
    
    return schedule

In [10]:
def fixDVOA(schedule):
    #import necessary files
    dvoa = pd.read_csv('dvoa.csv')
    dvoaID = pd.read_csv('dvoaConversion.csv')

    #convert TEAM IDs to match existing dataframes
    conID = []
    for team in dvoa['TEAM']:
        newTeam = dvoaID[dvoaID['TEAM'] == team]['newTeam'].iloc[0]
        conID.append(newTeam)
    
    dvoa['TEAM'] = conID
    return dvoa

In [11]:
def findDVOA(dvoa, schedule):
    dvoa['RRK'] = 64 - dvoa['RK']
    
    dvoaRKList = []
    dvoaRRKList = []
    dvoaWOList = []
    dvoaWDList = []
    
    for team in schedule['Winner Team']:
        for tm in dvoa['TEAM']:
            if team == tm:
                dvoaRK = dvoa[dvoa['TEAM'] == team]['RK'].iloc[0]
                dvoaRRK = dvoa[dvoa['TEAM'] == team]['RRK'].iloc[0]
                dvoaWO = dvoa[dvoa['TEAM'] == team]['OFF RANK'].iloc[0]
                dvoaWD = dvoa[dvoa['TEAM'] == team]['DEF RANK'].iloc[0]
                dvoaRKList.append(dvoaRK)
                dvoaRRKList.append(dvoaRRK)
                dvoaWOList.append(dvoaWO)
                dvoaWDList.append(dvoaWD)
                
    schedule['Winner DVOA'] = dvoaRKList
    schedule['Winner RDVOA'] = dvoaRRKList
    schedule['Winner OFF DVOA'] = dvoaWOList
    schedule['Winner DEF DVOA'] = dvoaWDList

    dvoaRKList = []
    dvoaRRKList = []
    dvoaWOList = []
    dvoaWDList = []
    for team in schedule['Loser Team']:
        for tm in dvoa['TEAM']:
            if team == tm:
                dvoaRK = dvoa[dvoa['TEAM'] == team]['RK'].iloc[0]
                dvoaRRK = dvoa[dvoa['TEAM'] == team]['RRK'].iloc[0]
                dvoaWO = dvoa[dvoa['TEAM'] == team]['OFF RANK'].iloc[0]
                dvoaWD = dvoa[dvoa['TEAM'] == team]['DEF RANK'].iloc[0]
                dvoaRKList.append(dvoaRK)
                dvoaRRKList.append(dvoaRRK)
                dvoaWOList.append(dvoaWO)
                dvoaWDList.append(dvoaWD)
                
    
    schedule['Loser DVOA'] = dvoaRKList
    schedule['Loser RDVOA'] = dvoaRRKList
    schedule['Loser OFF DVOA'] = dvoaWOList
    schedule['Loser DEF DVOA'] = dvoaWDList
    
    schedule['DVOA Delta'] = schedule['Winner DVOA'] - schedule['Loser DVOA']
    schedule['DVOA RDelta'] = schedule['Winner RDVOA'] - schedule['Loser RDVOA']
    schedule['DVOA Matchup'] = abs((schedule['Winner RDVOA'] - schedule['Loser RDVOA'])**2)
    
    schedule['WinO/LoseD DVOA'] = schedule['Winner OFF DVOA'] - schedule['Loser DEF DVOA']
    schedule['LoseO/WinD DVOA'] = schedule['Loser OFF DVOA'] - schedule['Winner DEF DVOA']
    
    return dvoa, schedule

In [12]:
def calcStats(fantasyTotal):
    #average fantasy PPG / salary
    fantasyTotal['FP/$'] = (fantasyTotal['DKPt'])/(fantasyTotal['Salary']*fantasyTotal['G'])

    #determine normal stats for the major catagories
    
    #create new list for each slot
    QBList = fantasyTotal[fantasyTotal['Position'] == 'QB'].sort_values(by=['FP/$'], ascending=False)
    RBList = fantasyTotal[fantasyTotal['Position'] == 'RB'].sort_values(by=['FP/$'], ascending=False)
    WRList = fantasyTotal[fantasyTotal['Position'] == 'WR'].sort_values(by=['FP/$'], ascending=False)
    TEList = fantasyTotal[fantasyTotal['Position'] == 'TE'].sort_values(by=['FP/$'], ascending=False)
    FlexList = pd.concat([RBList, WRList, TEList]).sort_values(by=['FP/$'], ascending=False)
    rosterList = [QBList, RBList, WRList, TEList]

    statsDict = {}
    for df in rosterList:
        avg = df['FP/$'].mean()
        med = df['FP/$'].median()
        std = df['FP/$'].std()
        maximum = df['FP/$'].max()
        minimum = df['FP/$'].min()
        name = df['Position'].iloc[0]
        statsDict[name] = ({'average' : avg, 'median': med, 'stdev': std, 'max': maximum, 'min': minimum})
    stats = pd.DataFrame(statsDict)
    
    return fantasyTotal, stats

In [13]:
def calcPlayerStats(fantasyTotal, schedule):
    statVal = []
    statVal2 = []
    statVal3 = []
    statVal4 = []
    statVal5 = []
    matchVal = []
    matchVal2 = []
    for player in fantasyTotal['Name']:
        tm = fantasyTotal[fantasyTotal['Name']==player]['Team'].iloc[0]
        for team in schedule['Winner Team']:    
            if team == tm:
                stat = schedule[schedule['Winner Team'] == tm]['RR WinO/LoseD'].iloc[0]
                stat2 = schedule[schedule['Winner Team'] == tm]['Winner SoS'].iloc[0]
                stat4 = schedule[schedule['Winner Team'] == tm]['DVOA Matchup'].iloc[0]
                wins = schedule[schedule['Winner Team'] == tm]['Winner Wins'].iloc[0]
                val = fantasyTotal[fantasyTotal['Name'] == player]['FP/$']
                stat3 = schedule[schedule['Winner Team'] == tm]['WinO/LoseD DVOA'].iloc[0]
                statVal.append(stat*(val.iloc[0]))
                statVal2.append(stat*stat2*(val.iloc[0]))
                statVal3.append(stat*wins*stat2*(val.iloc[0]))
                statVal4.append(stat*stat4*(val.iloc[0]))
                matchVal.append(stat)
                matchVal2.append(stat3*stat3)
                statVal5.append(stat3*(val.iloc[0]))
        for team in schedule['Loser Team']:
            if team == tm:
                stat = schedule[schedule['Loser Team'] == tm]['RR LoseO/WinD'].iloc[0]
                stat2 = schedule[schedule['Loser Team'] == tm]['Loser SoS'].iloc[0]
                stat4 = schedule[schedule['Loser Team'] == tm]['DVOA Matchup'].iloc[0]
                val = fantasyTotal[fantasyTotal['Name'] == player]['FP/$']
                wins = schedule[schedule['Loser Team'] == tm]['Loser Wins'].iloc[0]
                stat3 = schedule[schedule['Loser Team'] == tm]['LoseO/WinD DVOA'].iloc[0]
                statVal.append(stat*(val.iloc[0]))
                statVal2.append(stat*stat2*(val.iloc[0]))
                statVal3.append(stat*wins*stat2*(val.iloc[0]))
                statVal4.append(stat*stat4*(val.iloc[0]))
                matchVal.append(stat)
                matchVal2.append(stat3*stat3)
                statVal5.append(stat3*(val.iloc[0]))
    fantasyTotal['SoS Matchup'] = matchVal
    fantasyTotal['DVOA Matchup'] = matchVal2
    fantasyTotal['stat'] = statVal
    fantasyTotal['stat_sos'] = statVal2
    fantasyTotal['stat_sos_wins'] = statVal3
    fantasyTotal['stat_dvoa'] = statVal4
    fantasyTotal['stat_dvoa_delta'] = statVal5
    
    return fantasyTotal

In [14]:
def advStats(fantasyTotal, schedule):
    winRate = []
    
    for team in schedule['Winner Team']:
        wins = schedule[schedule['Winner Team'] == team]['Winner Wins'].iloc[0]
        loses = schedule[schedule['Winner Team'] == team]['Winner Loses'].iloc[0]
        games = wins + loses
        winPer = wins/games
        winRate.append(winPer)
    
    schedule['Winner Win Percentage'] = winRate
    winRate = []
    
    for team in schedule['Loser Team']:
        wins = schedule[schedule['Loser Team'] == team]['Loser Wins'].iloc[0]
        loses = schedule[schedule['Loser Team'] == team]['Loser Loses'].iloc[0]
        games = wins + loses
        winPer = wins/games
        winRate.append(winPer)
    
    schedule['Loser Win Percentage'] = winRate
    
    schedule['Winner Win SoS'] = schedule['Winner Win Percentage']*schedule['Winner SoS'] 
    schedule['Loser Win SoS'] = schedule['Loser Win Percentage']*schedule['Loser SoS']
    
    return schedule

In [15]:
week = pd.read_csv('seasonFantasy.csv')

week = addName(week)
week = addInjury(week)
week = addSalary(week)
week = dataCleanUp(week)
issue = issuesList(week)
week = issuesFix(week)
[week, weekStats] = calcStats(week)
sch = findSoS(week, 9)
dvoa = fixDVOA(sch)
[dvoa, sch] = findDVOA(dvoa, sch)
sch = scheduleAnalysis(week, sch)
week = calcPlayerStats(week, sch)
sch = advStats(week, sch)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

In [16]:
#need to fix SoS functions and scheduling data
#fix traded team values
#fix dvoa match and stat calcs

In [17]:
week.to_csv('analysis-week9.csv')
sch.to_csv('analysis-schedule9.csv')

In [26]:
week = week.sort_values(by=['stat_dvoa'], ascending=False)

In [27]:
QBList = week[week['Position'] == 'QB'].head(10)
QBList

,Rk,Player,Team,Position,Age,G,GS,Cmp,Att,Yds,...,Injury,Salary,FP/$,SoS Matchup,DVOA Matchup,stat,stat_sos,stat_sos_wins,stat_dvoa,stat_dvoa_delta
87,88,Ben Roethlisberger\RoetBe00,PIT,QB,38,7,7,167,246,1628,...,NA,6500.0,0.002785,3364,100,9.367446,4.224535,29.571742,7344.077785,-0.027846
3,4,Patrick Mahomes\MahoPa00,KAN,QB,25,8,8,190,284,2315,...,NA,8100.0,0.003304,2116,576,6.991290,2.772408,19.406857,1006.745778,-0.079296
4,5,Russell Wilson\WilsRu00,SEA,QB,32,7,7,183,256,2151,...,NA,7600.0,0.004098,2209,400,9.051917,3.655582,21.933492,579.322707,-0.081955
56,57,Deshaun Watson\WatsDe00,HOU,QB,25,7,7,166,239,2095,...,NA,7100.0,0.003330,1225,289,4.079225,2.610704,2.610704,493.586268,-0.056610
18,19,Josh Allen\AlleJo02,BUF,QB,24,8,8,186,277,2172,...,NA,7000.0,0.003475,1225,169,4.256875,1.915594,11.493562,272.440000,-0.045175
350,351,Chad Henne\HennCh01,KAN,QB,35,2,0,5,6,30,...,NA,4100.0,0.000854,2116,576,1.806341,0.716308,5.014155,260.113171,-0.020488
349,350,Ben DiNucci\DiNuBe00,DAL,QB,24,3,1,23,43,219,...,NA,4800.0,0.000625,256,576,0.160000,0.077333,0.154667,125.440000,0.015000
68,69,Ryan Tannehill\TannRy00,TEN,QB,32,7,7,157,233,1823,...,NA,6300.0,0.003676,1156,4,4.249152,1.869627,9.348134,106.228798,-0.007351
83,84,Teddy Bridgewater\BridTe00,CAR,QB,28,8,8,184,257,2106,...,NA,6000.0,0.002985,225,1,0.671719,0.347055,1.041164,96.727500,-0.002985
78,79,Justin Herbert\HerbJu00,LAC,QB,22,6,6,153,227,1820,...,NA,6800.0,0.003897,900,400,3.507353,1.821126,3.642251,87.683824,-0.077941


In [28]:
QBList = week[week['Position'] == 'QB'].tail(10)
QBList

,Rk,Player,Team,Position,Age,G,GS,Cmp,Att,Yds,...,Injury,Salary,FP/$,SoS Matchup,DVOA Matchup,stat,stat_sos,stat_sos_wins,stat_dvoa,stat_dvoa_delta
131,132,Drew Lock\LockDr00,DEN,QB,24,5,5,83,143,927,...,NA,5200.0,0.002200,1089,64,2.395800,1.336119,4.008358,9.583200,0.017600
79,80,Lamar Jackson\JackLa00,BAL,QB,23,7,7,115,190,1343,...,NA,6900.0,0.003184,576,289,1.834137,0.971014,4.855068,7.336547,0.054133
88,89,Ryan Fitzpatrick\FitzRy00,MIA,QB,38,6,6,138,197,1535,...,NA,5000.0,0.004430,400,121,1.772000,0.802415,3.209660,7.088000,0.048730
7,8,Kyler Murray\MurrKy00,ARI,QB,23,7,7,169,253,1847,...,NA,7800.0,0.003839,441,1,1.692923,0.638839,3.194194,6.771692,-0.003839
318,319,Brett Rypien\RypiBr00,DEN,QB,24,3,1,27,40,295,...,NA,4000.0,0.001275,1089,64,1.388475,0.774342,2.323025,5.553900,0.010200
93,94,Philip Rivers\RivePh00,IND,QB,39,7,7,161,231,1860,...,NA,5600.0,0.002837,441,289,1.251000,0.433038,2.165192,5.004000,0.048224
422,423,Alex Smith\SmitAl03,WAS,QB,36,1,0,9,17,37,...,NA,4800.0,0.000333,361,169,0.120333,0.059031,0.118063,4.332000,0.004333
372,373,Tua Tagovailoa\TagoTu00,MIA,QB,22,2,1,14,24,102,...,NA,5500.0,0.000645,400,121,0.258182,0.116913,0.467650,1.032727,0.007100
431,432,Jacoby Brissett\BrisJa00,IND,QB,28,2,0,2,4,17,...,NA,4400.0,0.000080,441,289,0.035080,0.012143,0.060715,0.140318,0.001352
521,522,Chris Streveler\StreCh00,ARI,QB,25,3,0,0,0,0,...,NA,4100.0,0.000024,441,1,0.010756,0.004059,0.020295,0.043024,-0.000024


In [29]:
RBList = week[week['Position'] == 'RB'].head(10)
RBList

,Rk,Player,Team,Position,Age,G,GS,Cmp,Att,Yds,...,Injury,Salary,FP/$,SoS Matchup,DVOA Matchup,stat,stat_sos,stat_sos_wins,stat_dvoa,stat_dvoa_delta
24,25,James Conner\ConnJa00,PIT,RB,25,7,6,0,0,0,...,NA,6900.0,0.002408,3364,100,8.100066,3.652971,25.570797,6350.451942,-0.024079
212,213,Benny Snell Jr.\SnelBe00,PIT,RB,22,7,0,0,0,0,...,NA,4000.0,0.001168,3364,100,3.928671,1.771754,12.402276,3080.078400,-0.011679
353,354,Anthony McFarland Jr.\McFaAn00,PIT,RB,21,5,0,0,0,0,...,NA,4000.0,0.000450,3364,100,1.513800,0.682694,4.778859,1186.819200,-0.004500
26,27,Clyde Edwards-Helaire\EdwaCl00,KAN,RB,21,8,8,0,0,0,...,NA,6100.0,0.002471,2116,576,5.229295,2.073686,14.515802,753.018492,-0.059311
17,18,Ezekiel Elliott\ElliEz00,DAL,RB,25,8,8,0,0,0,...,questionable,6600.0,0.002805,256,576,0.718061,0.347063,0.694125,562.959515,0.067318
252,253,Le'Veon Bell\BellLe00,KAN,RB,28,4,2,0,0,0,...,NA,5100.0,0.001373,2116,576,2.904314,1.151711,8.061974,418.221176,-0.032941
52,53,David Johnson\JohnDa08,HOU,RB,29,7,7,0,0,0,...,NA,5600.0,0.002559,1225,289,3.134375,2.006000,2.006000,379.259375,-0.043497
14,15,James Robinson\RobiJa00,JAX,RB,22,7,7,0,0,0,...,NA,7000.0,0.002869,1024,81,2.938253,1.319216,1.319216,355.528620,-0.025824
458,459,Jaylen Samuels\SamuJa00,PIT,RB,24,7,0,0,0,0,...,NA,4000.0,0.000118,3364,100,0.396471,0.178801,1.251606,310.833600,-0.001179
236,237,Darrel Williams\WillDa10,KAN,RB,25,8,0,0,0,0,...,NA,4000.0,0.000928,2116,576,1.963912,0.778793,5.451550,282.803400,-0.022275


In [30]:
WRList = week[week['Position'] == 'WR'].head(10)
WRList

,Rk,Player,Team,Position,Age,G,GS,Cmp,Att,Yds,...,Injury,Salary,FP/$,SoS Matchup,DVOA Matchup,stat,stat_sos,stat_sos_wins,stat_dvoa,stat_dvoa_delta
30,31,Chase Claypool\ClayCh01,PIT,WR,22,7,2,0,0,0,...,NA,5700.0,0.002672,3364,100,8.987529,4.053199,28.372395,7046.222596,-0.026717
194,195,James Washington\WashJa00,PIT,WR,24,7,2,0,0,0,...,NA,3000.0,0.002405,3364,100,8.089619,3.648260,25.537817,6342.261333,-0.024048
160,161,Diontae Johnson\JohnDi01,PIT,WR,24,6,6,0,0,0,...,NA,5000.0,0.002307,3364,100,7.759627,3.499439,24.496076,6083.547307,-0.023067
112,113,JuJu Smith-Schuster\SmitJu00,PIT,WR,24,7,7,0,0,0,...,NA,6200.0,0.002226,3364,100,7.487613,3.376767,23.637366,5870.288516,-0.022258
330,331,Ray-Ray McCloud\McClRa00,PIT,WR,24,7,1,0,0,0,...,NA,3000.0,0.000729,3364,100,2.450914,1.105314,7.737200,1921.516800,-0.007286
9,10,Tyreek Hill\HillTy00,KAN,WR,26,8,8,0,0,0,...,NA,7100.0,0.002555,2116,576,5.405486,2.143555,15.004883,778.389972,-0.061310
108,109,Mecole Hardman\HardMe00,KAN,WR,22,8,5,0,0,0,...,NA,4800.0,0.002117,2116,576,4.479969,1.776539,12.435775,645.115500,-0.050812
44,45,Amari Cooper\CoopAm00,DAL,WR,26,8,7,0,0,0,...,NA,5600.0,0.002904,256,576,0.743429,0.359324,0.718648,582.848000,0.069696
58,59,CeeDee Lamb\LambCe00,DAL,WR,21,8,6,0,0,0,...,NA,4900.0,0.002824,256,576,0.722939,0.349420,0.698841,566.784000,0.067776
218,219,Demarcus Robinson\RobiDe01,KAN,WR,26,8,4,0,0,0,...,NA,3200.0,0.001781,2116,576,3.769125,1.494653,10.462571,542.754000,-0.042750


In [31]:
TEList = week[week['Position'] == 'TE'].head(10)
TEList

,Rk,Player,Team,Position,Age,G,GS,Cmp,Att,Yds,...,Injury,Salary,FP/$,SoS Matchup,DVOA Matchup,stat,stat_sos,stat_sos_wins,stat_dvoa,stat_dvoa_delta
173,174,Eric Ebron\EbroEr00,PIT,TE,27,7,6,0,0,0,...,NA,4500.0,0.002105,3364,100,7.080419,3.193130,22.351911,5551.048533,-0.021048
367,368,Vance McDonald\McDoVa00,PIT,TE,30,7,5,0,0,0,...,questionable,2500.0,0.000794,3364,100,2.671977,1.205009,8.435065,2094.830080,-0.007943
6,7,Travis Kelce\KelcTr00,KAN,TE,31,8,8,0,0,0,...,NA,7200.0,0.002552,2116,576,5.400208,2.141462,14.990233,777.630000,-0.061250
144,145,Dalton Schultz\SchuDa00,DAL,TE,24,8,6,0,0,0,...,NA,3600.0,0.002726,256,576,0.697778,0.337259,0.674519,547.057778,0.065417
161,162,Darren Fells\FellDa01,HOU,TE,34,7,7,0,0,0,...,NA,4200.0,0.001888,1225,289,2.312500,1.480000,1.480000,279.812500,-0.032092
265,266,Will Dissly\DissWi00,SEA,TE,24,7,4,0,0,0,...,NA,2700.0,0.001646,2209,400,3.634915,1.467947,8.807679,232.634582,-0.032910
285,286,James O'Shaughnessy\O'ShJa00,JAX,TE,28,7,7,0,0,0,...,NA,2900.0,0.001611,1024,81,1.649498,0.740591,0.740591,199.589202,-0.014498
304,305,Jacob Hollister\HollJa03,SEA,TE,27,7,0,0,0,0,...,NA,2500.0,0.001017,2209,400,2.246869,0.907389,5.444335,143.799589,-0.020343
346,347,Blake Bell\BellBl00,DAL,TE,29,8,2,0,0,0,...,NA,2500.0,0.000605,256,576,0.154880,0.074859,0.149717,121.425920,0.014520
221,222,Tyler Kroft\KrofTy00,BUF,TE,28,8,4,0,0,0,...,NA,3200.0,0.001477,1225,169,1.808789,0.813955,4.883730,115.762500,-0.019195


In [32]:
weekStats

,QB,RB,WR,TE
average,0.002454,0.001252,0.001719,0.001417
median,0.002881,0.001155,0.001977,0.001241
stdev,0.001354,0.001001,0.000914,0.000958
max,0.004430,0.003433,0.003183,0.003619
min,0.000024,0.000004,0.000038,0.000091


In [34]:
sch.sort_values(by=['DVOA Matchup'], ascending=False)

,Week,Day,Date,Time,Winner/tie,Unnamed: 5,Loser/tie,Unnamed: 7,PtsW,PtsL,...,ReverseRank Lose Off,ReverseRank Lose Def,WinOff/LoseDef Delta,LoseOff/WinDef Delta,RR LoseO/WinD,RR WinO/LoseD,Winner Win Percentage,Loser Win Percentage,Winner Win SoS,Loser Win SoS
129,9,Sun,November 8,4:25PM,Pittsburgh Steelers,@,Dallas Cowboys,preview,NaN,NaN,...,44,0,-26,16,256,3364,1.000000,0.250000,0.450980,0.120833
132,9,Mon,November 9,8:15PM,New England Patriots,@,New York Jets,preview,NaN,NaN,...,32,1,-1,21,121,1089,0.285714,0.000000,0.186813,0.000000
131,9,Sun,November 8,8:20PM,New Orleans Saints,@,Tampa Bay Buccaneers,preview,NaN,NaN,...,61,23,1,-13,2025,961,0.714286,0.750000,0.384615,0.355932
124,9,Sun,November 8,1:00PM,Carolina Panthers,@,Kansas City Chiefs,preview,NaN,NaN,...,63,26,17,-14,2116,225,0.375000,0.875000,0.193750,0.346983
120,9,Sun,November 8,1:00PM,Houston Texans,@,Jacksonville Jaguars,preview,NaN,NaN,...,37,4,-3,0,1024,1225,0.142857,0.142857,0.091429,0.064140
123,9,Sun,November 8,1:00PM,Seattle Seahawks,@,Buffalo Bills,preview,NaN,NaN,...,50,13,-15,-3,1225,2209,0.857143,0.750000,0.346154,0.337500
121,9,Sun,November 8,1:00PM,New York Giants,@,Washington Football Team,preview,NaN,NaN,...,33,24,21,13,361,121,0.125000,0.285714,0.069672,0.140162
128,9,Sun,November 8,4:05PM,Las Vegas Raiders,@,Los Angeles Chargers,preview,NaN,NaN,...,42,18,4,2,900,784,0.571429,0.285714,0.359788,0.148352
122,9,Sun,November 8,1:00PM,Chicago Bears,@,Tennessee Titans,preview,NaN,NaN,...,56,19,13,-2,1156,361,0.625000,0.714286,0.307377,0.314286
119,9,Thu,November 5,8:20PM,Green Bay Packers,@,San Francisco 49ers,boxscore,34.0,17.0,...,59,8,-22,-16,2304,2916,0.714286,0.500000,0.294118,0.220339
